In [1]:
# STEP1: Before running the protocol from a Jupyter Notebook, you need to upload its corresponding calibration protocol in the python format (.py) in the Opentrons App which consists of all the pipettes and labware used in the corresponding protocol. 
# Once uploaded, the Opentrons App will recognise which pipettes to be attached and can calibrated as described in (Jones, n.d.). The Opentrons App will then display a layout of the deck and the labware (e.g., tips racks, well plates, etc…) which can
# now be positioned onto the deck. Calibration should be done before running code on Jupyter Notebook since this is not supported by Opentrons-2 (OT-2) at the moment. Otherwise, the existing calibration from previous calibrations in OT-2 will be used.

# STEP2: Each falcon tube should be filled with 30 ml of PBS for the protocol to run properly.

# STEP3: Jupyter Notebook is pre-installed in each Opentron robot, which can be accessed through your preferred web browser. For more detailed instructions, please refer to the README document on the same repository as this file.


import opentrons.execute
from opentrons import protocol_api


protocol = opentrons.execute.get_protocol_api('2.11')
protocol.home()
metadata = {'apiLevel': '2.11'}

# Function for obtaining valid numerical input. If input is out of predefined range/ invalid, error message arises
def get_numeric_input(message, ntype=int, min_value=0, max_value=100):

    is_valid = False
    while not is_valid:
        try:
            result = ntype(input(message))
            if result == 'q': # Users can enter 'q' to exit, otherwise they will keep getting asked for input until they enter something correct.
                raise Exception('"q" was entered to quit')
            if result < min_value or result > max_value:
                print("Invalid value entered. Value must be in the range [{}, {}]".format(min_value, max_value))
            else:
                is_valid = True
        except ValueError:
            print("Invalid value entered. You must enter a {}.".format(ntype.__name__))
    return result


#Obtaining user input for important parameters 
repeat = get_numeric_input("How many technical replicates would you like to conduct? (maximum: 24)", max_value=24)
promoter = get_numeric_input("How many promoters are you testing?",max_value = 11)
inducer = get_numeric_input("How many inducers are you working with? (maximum: 2)", max_value=2)
conc = get_numeric_input("How many concentrations will you be testing for each inducer? (maximum: 12)", max_value=12)
vol_ind = get_numeric_input("What will be the volume of inducer that goes into the well-plate (in uL)? recommended: 1uL")
vol_cell = get_numeric_input("What will be the volume of cell culture that goes into the well-plate (in uL)? recommended: 60uL")
vol_oil = get_numeric_input("What will be the volume of oil that goes on top of each well (in uL)? recommended: 20uL")
temperature = get_numeric_input("What temperature would you like to test the protein expression at (in celcius) ? recommended: 30oC")


#Checking if the number of samples requested by the user can fit on one 384 well plate 
if promoter*inducer*conc*repeat > 384: 
    print("Too many samples - cannot be executed in a single protocol") 



#Loading all the labware and instruments
tiprack1 = protocol.load_labware('opentrons_96_tiprack_300ul', 11)
tiprack2 = protocol.load_labware('opentrons_96_tiprack_20ul', 10)


left = protocol.load_instrument(
            'p300_single_gen2', 'left', tip_racks=[tiprack1])
right = protocol.load_instrument(
            'p20_single_gen2', 'right', tip_racks=[tiprack2])


Reservoir1 = protocol.load_labware('nest_12_reservoir_15ml',7) #Inducer 1 
Reservoir2 = protocol.load_labware('nest_12_reservoir_15ml',1) #Inducer 2  


temp_mod = protocol.load_module('temperature module', 4)

plate = temp_mod.load_labware('corning_384_wellplate_112ul_flat') #Plate will be on top of the temperature module 

tuberack1 = protocol.load_labware('opentrons_6_tuberack_falcon_50ml_conical',8) #First set of cell cultures 
tuberack2 = protocol.load_labware('opentrons_6_tuberack_falcon_50ml_conical',5)#Second set of cell cultures with mineral oil at bottom right


#Custom-made function that helps distribute volumes and blows out excess into the source well instead of trash 
def onetomany(vol,source,spots,cap,top = False): 
    disposal = cap*0.1 #setting disposal volume as 10% of the pipette capacity 
    num_spots = len(spots) #number of wells to pipette 
    
    if cap == 300: #left pipette 
        left.pick_up_tip()
        if (vol*num_spots)+disposal  <= cap: #Checking if the total volume to be dispensed is greater than the pipette capacity 
            left.aspirate((vol*num_spots)+disposal,source.top(-90))
            for i in range(num_spots):
                if top == True: #Dispensing at the top of the well whenever specified
                    left.dispense(vol,plate[spots[i]].top())
                else: 
                    left.dispense(vol,plate[spots[i]])
            left.blow_out(source) #Blowing out the disposal volume at the source well 
            left.drop_tip()
        else:    
            dispense_at_once = int((cap-disposal)//(vol)) #Number of dispenses in each loop 
            i = 0 #Initialising the number of wells filled 
            while i < len(spots): 
                left.aspirate((vol*dispense_at_once)+disposal,source.top(-90))
                for j in range(dispense_at_once): 
                    if top == True: #Dispensing at the top of the well whenever specified
                        left.dispense(vol,plate[spots[i]].top())
                    else: 
                        left.dispense(vol,plate[spots[i]])
                    i += 1
                    if i == len(spots): #Checking if all the wells have been filled
                        break 
                left.blow_out(source) #Blowing out the disposal volume at the source well 
            left.drop_tip()
            
    elif cap == 20: #right pipette 
        right.pick_up_tip()
        if (vol*num_spots)+disposal  <= cap: #Checking if the total volume to be dispensed is greater than the pipette capacity
            right.aspirate((vol*num_spots)+disposal,source)
            for i in range(num_spots):
                if top == True: #Dispensing at the top of the well whenever specified 
                    right.dispense(vol,plate[spots[i]].top())
                else: 
                    right.dispense(vol,plate[spots[i]])
            right.blow_out(source) #Blowing out the disposal volume at the source well 
            right.drop_tip()
        else: 
            dispense_at_once = int((cap-disposal)//(vol)) #Number of dispenses in each loop
            i = 0  #Initialising the number of wells filled 
            while i < len(spots): 
                right.aspirate((vol*dispense_at_once)+disposal,source)
                for j in range(dispense_at_once): 
                    if top == True: #Dispensing at the top of the well whenever specified
                        right.dispense(vol,plate[spots[i]].top())
                    else: 
                        right.dispense(vol,plate[spots[i]])
                    i += 1
                    if i == len(spots): #Checking if all the wells have been filled 
                        break 
                right.blow_out(source)#Blowing out the disposal volume at the source well 
            right.drop_tip()


            
            
#Defining locations on the 384 well plate 
rows = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P']
cols = list(range(1,25))
positions = []
for x in range(16):
    for y in range(24): 
        positions.append(rows[x]+str(cols[y]))
        
#Creating a dictionary to help keep track of the contents of each well 
mydict = {position : "NA" for position in positions} 


#Defining locations on the falcon tube rack 
tubes = []
for p in range(2):
    for q in range(3): 
        tubes.append(rows[p]+str(cols[q]))



## STEP 1: Pipetting cell cultures 
top_prom = int(len(cols)/repeat) #First set of samples starting from the top left of the well plate 

#Identifying the wells that are assigned to each combination of inducer,promoter and concentration 
for j in range(1,(promoter*inducer)+1): 
    if j > (top_prom): 
        spots = []
        for k in range(conc): 
            row = rows[k+conc] #Each concentration is on a different row 
            for l in range(repeat): 
                column = cols[l+((j-top_prom)-1)*repeat] #Each repeat is on a different column
                position = row + str(column)
                spots.append(position) 

    else: #Moving to the wells at the bottom of the plate for additional samples 
        spots = [] 
        for k in range(conc): 
            row = rows[k]
            for l in range(repeat): 
                column = cols[l+(j-1)*repeat]
                position = row + str(column)
                spots.append(position)
    
    if j > promoter: 
        m = j - promoter 
    else:  m = j 
        
    if m in range(1,7): 
         cell_source = tuberack1[tubes[m-1]]
    elif m in range(7,14): #When using more than 6 promoters, the pipette will move to the second falcon tube rack 
        cell_source = tuberack2[tubes[m-1]]
        
    onetomany(vol_cell,cell_source,spots,300)#Distributing cell cultures to the different wells 
    

## STEP 2: Pipetting inducers  

for b in range(conc): 
    spots = []
    for c in range(1,(promoter*inducer)+1): 
        if c > (top_prom): 
            row = rows[b+conc]
            for d in range(repeat): 
                column = cols[d+((c-top_prom)-1)*repeat]
                spots.append(row + str(column)) 
        else: 
            row = rows[b]
            for d in range(repeat): 
                column = cols[d+(c-1)*repeat]
                spots.append(row + str(column))

    inducer1 = spots[:promoter*repeat]
    inducer2 = spots[promoter*repeat:]

    for a in range(inducer):     
        for e in range(1,promoter+1): 
            start = (e-1)*repeat 
            end = start+repeat
            if a == 0: 
                loc = inducer1[start:end]
                for q in range(len(loc)): 
                    mydict[loc[q]] = str("P")+str(e)+"_"+str("I")+str(a+1)+"_"+str("C")+str(b+1) #Updating the contents of the wells 
                
                #Distributing different concentrations of Inducer 1 to the different wells    
                source_ind = Reservoir1["A"+str(b+1)]
                right.distribute(vol_ind,[Reservoir1.wells("A"+str(b+1))],[plate.wells_by_name()[well_name] for well_name in loc],disposal_volume= 2)
            else: 
                loc = inducer2[start:end]
                for q in range(len(loc)): 
                    mydict[loc[q]] = str("P")+str(e)+"_"+str("I")+str(a+1)+"_"+str("C")+str(b+1)#Updating the contents of the wells
                    
                #Distributing different concentrations of Inducer 2 to the different wells 
                source_ind = Reservoir2["A"+str(b+1)]
                right.distribute(vol_ind,[Reservoir2.wells("A"+str(b+1))],[plate.wells_by_name()[well_name] for well_name in loc],disposal_volume= 2)  
                

## STEP 3:  Mixing the inducers and cells in each well  

for j in range(1,(promoter*inducer)+1): 
    if j > (top_prom): 
        for k in range(conc): 
            row = rows[k+conc]
            left.pick_up_tip()
            for l in range(repeat): 
                column = cols[l+((j-top_prom)-1)*repeat]
                loc = row+str(column)
                left.mix(2, 64,plate[loc])
            left.drop_tip()

    else: 
        for k in range(conc): 
            row = rows[k]
            left.pick_up_tip()
            for l in range(repeat): 
                column = cols[l+(j-1)*repeat]
                loc = row+str(column)
                left.mix(2, 64,plate[loc])
            left.drop_tip()


## STEP 4: Adding a layer of mineral oil on the top of each well to prevent evaporation 

spots = []
for b in range(len(positions)): 
    well = positions[b]
    if mydict[well] != "NA": 
        spots.append(well)

print(spots)
right.flow_rate.aspirate = 50 #Changing the speed of the pipette for a more viscous liquid 
right.flow_rate.dispense = 50 #Changing the speed of the pipette for a more viscous liquid 
source = tuberack2.wells_by_name()['B3']

onetomany(vol_oil,source,spots,300,True)


## STEP 5: Setting the right temperature for protein expression 
temp_mod.set_temperature(temperature)






/Users/suhasiniiyer/.opentrons/robot_settings.json not found. Loading defaults
Failed to initialize character device, will not be able to control gpios (lights, button, smoothiekill, smoothie reset). Only one connection can be made to the gpios at a time. If you need to control gpios, first stop the robot server with systemctl stop opentrons-robot-server. Until you restart the server with systemctl start opentrons-robot-server, you will be unable to control the robot using the Opentrons app.
This is intended to run on a robot, and while it can connect to a smoothie via a usb/serial adapter unexpected things using gpios (such as smoothie reset or light management) will fail. If you are seeing this message and you are running on a robot, you need to set the RUNNING_ON_PI environmental variable to 1.
Failed to initiate aionotify, cannot watch modules or door, likely because not running on linux
Motor driver could not connect, reprogramming if possible
Traceback (most recent call last):
  

ThreadManagerException: Failed to create Managed Object

In [ ]:
## WRITING THE PLATE LAYOUT ONTO AN EXCEL FILE FOR USER REFERENCE 
from openpyxl import load_workbook
wb = load_workbook("plate_layout.xlsx")
sheet = wb.active

rows = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P']
columns = list(range(1,25))
positions = []
for x in range(16):
    for y in range(24): 
        positions.append(rows[x]+str(columns[y]))

#Naming the outlines of the plate 
for x in range(1,17):  
        well = sheet.cell(row = x+1, column = 1)
        well.value = rows[x-1]
        
for y in range(1,25):  
        well = sheet.cell(row = 1, column = y+1)
        well.value = cols[y-1]

#Writing the contents of the individual wells 
r = 2 
c = 2    
for key in mydict.keys():
    well = sheet.cell(row = r, column = c)
    well.value = mydict[key]
    c += 1
    if c == 26: 
        r+=1 
        c = 2
    if r == 18: 
        break 
wb.save("plate_layout.xlsx")
        
        


In [ ]:
## DROPPING THE TIP FROM THE RIGHT PIPETTE 
import opentrons.execute
from opentrons import protocol_api


protocol = opentrons.execute.get_protocol_api('2.11')
protocol.home()
metadata = {'apiLevel': '2.11'}
right.drop_tip()


In [ ]:
## DROPPING THE TIP FROM THE LEFT PIPETTE 
import opentrons.execute
from opentrons import protocol_api


protocol = opentrons.execute.get_protocol_api('2.11')
protocol.home()
metadata = {'apiLevel': '2.11'}
left.drop_tip()
